In [80]:
import hashlib
import sys
import os
import pandas as pd
from unidecode import unidecode

In [81]:
# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [82]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [83]:
def processar_disciplina(disciplina):
    """
    Processa uma string de detalhes de disciplina, dividindo em componentes.

    Args:
        disciplina (str): String contendo os detalhes da disciplina.

    Returns:
        tuple: Uma tupla contendo os detalhes processados da disciplina.
    """
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)


def obter_disciplina_e_codigo(x):
    """
    Extrai código e nome da disciplina a partir de uma lista de strings.

    Args:
        x (list): Lista contendo os detalhes da disciplina.

    Returns:
        tuple: Uma tupla com o código e o nome da disciplina, ou NaN se não disponível.
    """
    if x[0] == "nan":
        return np.nan
    try:
        return (
            x[1].split()[0].replace(" ", ""),
            " ".join(x[1].split()[1:]),
        )
    except IndexError:
        print("Erro de índice para:", x)
        return None

In [84]:
df_silver = extrair_dados_database("alunos_ufrj.parquet", "silver")

In [85]:
# Remove as linhas onde a coluna 'disciplinasCursadas' tem NaN ou None e obtém as disciplinas únicas
df_silver = df_silver.dropna(subset=["disciplinasCursadas"])[["disciplinasCursadas"]].drop_duplicates()

In [86]:
# Processa a coluna 'disciplinasCursadas' para separar em periodo, disciplina, grau, situação
df_silver["disciplinasCursadas"] = df_silver["disciplinasCursadas"].apply(
    lambda row: [processar_disciplina(item) for item in row.split("\n")]
)

# Explode a coluna 'disciplinasCursadas' para dividir as listas de disciplinas em linhas separadas
df_exploded = df_silver.explode("disciplinasCursadas")

# Processa os detalhes das disciplinas
df_exploded["disciplinasCursadas"] = df_exploded["disciplinasCursadas"].apply(
    obter_disciplina_e_codigo
)

# Remove disciplinas duplicadas
df_exploded.drop_duplicates(inplace=True)

In [87]:
# Cria DataFrame somente com codigo e nome da disciplina
d_disciplina = pd.DataFrame(
    df_exploded["disciplinasCursadas"].tolist(),
    index=df_exploded.index,
    columns=["codDisciplina", "nomeDisciplina"],
)

In [88]:
# Remover acentos e cedilhas
d_disciplina['nomeDisciplina'] = d_disciplina['nomeDisciplina'].apply(unidecode)
d_disciplina = d_disciplina.drop_duplicates()

In [89]:
# Dicionário de substituições de disciplinas
dicionario_substituicoes = {
    'Top Esp em Eng de Softaware': 'Top Esp em Eng de Software',
    'Fisico-quimica I': 'Fisico-Quimica I',
    'Topicos Especiais em Redes': 'Topicos Especiais em Redes I',
    # Adicione mais substituições conforme necessário
}

# Função para substituir frases com base no dicionário
def substituir_disciplina(frase):
    return dicionario_substituicoes.get(frase, frase)

d_disciplina['nomeDisciplina'] = d_disciplina['nomeDisciplina'].apply(substituir_disciplina)
d_disciplina.drop_duplicates(subset='nomeDisciplina', inplace=True)

In [75]:
# Criação do sk_d_disciplina usando SHA-256
d_disciplina["SK_D_DISCIPLINA"] = d_disciplina.apply(
    lambda x: hashlib.sha256(
        str(x["codDisciplina"] + x["nomeDisciplina"]).encode()
    ).hexdigest(),
    axis=1,
)

# Padronizando nome das colunas
d_disciplina = d_disciplina.rename(
    columns={
        "codDisciplina": "CD_DISCIPLINA",
        "nomeDisciplina": "DS_NOME_DISCIPLINA",
    }
)

In [76]:
# Reordenação das colunas
colunas_disciplina = ["SK_D_DISCIPLINA", "CD_DISCIPLINA", "DS_NOME_DISCIPLINA"]
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)

In [78]:
# d_disciplina[d_disciplina['CD_DISCIPLINA'].duplicated(keep=False)].sort_values('CD_DISCIPLINA')

In [79]:
# Salvar dados na camada Gold
carregar_dados_database(d_disciplina, "d_disciplina.parquet", "gold")